<center>
       <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="500" alt="cognitiveclass.ai logo">
</center>



# Financial services. 

# Lab 2. 

# Improving the cryptocurrency pair dataset for effective analysis (on the example BTC/USD and technical financial indicators ATR, OBV, ADV, RSI, AD)


Estimated time needed: **30** minutes

 The tasks:
* To find empty cells and handle missing values;
* Analyze data format, find the wrong format and correct data format;
* Standardize and normalize data series.

## Objectives

After completing this lab you will be able to:

*   Handle missing values
*   Correct data format
*   Standardize and normalize data


## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">
<ul>
    <li>Identify and handle missing values
        <ul>
            <li>Identify missing values</li>
            <li>Deal with missing values</li>
            <li>Correct data format</li>
        </ul>
    </li>
    <li>Data standardization</li>
    <li>Data normalization (centering/scaling)</li>
    <li>Binning</li>
    <li>Indicator variable</li>
    <li>Resample time-series data</li>
</ul>

</div>

<hr>


## What is the purpose of data wrangling?


Data wrangling is the process of converting data from the initial format to a format that may be better for analysis.


### What is the norm to open the trade?


### Import data
<p>
You can find the "Trading Dataset" from the following link: <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-GPXX0ZDWEN/BTCBUSD.csv">https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-GPXX0ZDWEN/BTCBUSD.csv</a>. 
We will be using this dataset throughout this course.
</p>


#### Import pandas 


If you run the lab locally using Anaconda, you can load the correct library and versions by uncommenting the following:


In [1]:
#If you run the lab locally using Anaconda, you can load the correct library and versions by uncommenting the following:
#install specific version of libraries used in lab
#! mamba install pandas==1.3.3
#! mamba install numpy=1.21.2
! conda install scikit-learn -y

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 23.3.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - scikit-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2023.01.10 |       h06a4308_0         120 KB
    certifi-2022.12.7          |   py37h06a4308_0         150 KB
    joblib-1.1.1               |   py37h06a4308_0         379 KB
    openssl-1.1.1t             |       h7f8727e_0         3.7 MB
    scikit-learn-1.0.2         |   py37h51133e4_1         5.5 MB
  

In [2]:
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
import requests
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline
import matplotlib as plt
from matplotlib import pyplot

import warnings
warnings.filterwarnings("ignore")

#set precision 2
pd.options.display.float_format = '{:.2f}'.format

## Reading the dataset from the URL and adding the related headers


First, we assign the URL of the dataset to "filename".


This dataset was hosted on IBM Cloud object. Click <a href="https://cocl.us/corsera_da0101en_notebook_bottom?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDA0101ENSkillsNetwork20235326-2021-01-01">HERE</a> for free storage.


In [3]:
filename = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-GPXX0ZDWEN/BTCBUSD.csv"

Then, we create a Python list <b>headers</b> containing name of headers.


In [4]:
headers = ["Time","Open","High","Low","Close","Volume","Rec_count","Avg_price","ATR","OBV","Gain","Loss","Ema_gain","Ema_loss",
          "RS","RSI_14","ADTV","MFI","MFV","ADL"]

Use the Pandas method <b>read_csv()</b> to load the data from the web address. Set the parameter  "names" equal to the Python list "headers".


In [5]:
#our work dataframe
df = pd.read_csv(filename, low_memory=False)
#dataframe for calculating our accuracy
newdf = pd.read_csv(filename, low_memory=False)

df.columns = headers
newdf.columns = headers

Use the method <b>head()</b> to display the first five rows of the dataframe.


In [6]:
# To see what the data set looks like, we'll use the head() method.
df.head()

,Time,Open,High,Low,Close,Volume,Rec_count,Avg_price,ATR,OBV,Gain,Loss,Ema_gain,Ema_loss,RS,RSI_14,ADTV,MFI,MFV,ADL
0,2022-11-11 14:38:00,16540.10,16540.10,16486.50,16486.50,304.95,97,16516.81,NaN,0.00,0.00,53.60,NaN,NaN,NaN,NaN,140.69,-1.00,-304.95,-304.95
1,2022-11-11 14:39:00,16489.30,16493.30,16456.20,16462.10,319.26,103,16473.84,NaN,-319.26,0.00,27.20,NaN,NaN,NaN,NaN,140.69,-0.68,-217.72,-522.67
2,2022-11-11 14:40:00,16465.50,16496.00,16447.40,16492.90,290.90,99,16478.64,NaN,-28.36,27.40,0.00,NaN,NaN,NaN,NaN,140.69,0.87,253.79,-268.88
3,2022-11-11 14:41:00,16492.90,16547.10,16472.50,16543.00,314.05,99,16516.50,NaN,285.69,50.10,0.00,NaN,NaN,NaN,NaN,140.69,0.89,279.53,10.65
4,2022-11-11 14:42:00,16540.20,16547.10,16498.20,16498.30,226.95,99,16527.09,NaN,58.74,0.00,41.90,NaN,NaN,NaN,NaN,140.69,-1.00,-226.02,-215.37


## Making some values wrong


To test different methods of dealing with errors in the dataset, we specifically make three types of errors: character, negative values, and missing data.


To have something to test functions, we need to replace some values:


In [7]:
l1 = 0.07 # probabilities of "?" values
l2 = 0.07 # probabilities of Text values
l3 = 0.07 # probabilities of  Negativevalues

m = np.random.rand(len(df))
mask1 = m < l1 # creation mask for "?"
m = np.random.rand(len(df))
mask2 = m < l2 # creation mask for Text
m = np.random.rand(len(df))
mask3 = m < l3 # creation mask for Negative

#replace values in our dataframe
df.loc[mask1, 'Rec_count'] = "?"
df.loc[mask2, 'Volume'] = "Error"
df.loc[mask3, 'Open'] = - df.loc[mask3, 'Open']

#save dataframe 
df.to_csv("btc.csv", index=False)
print(df)

                      Time      Open     High      Low    Close Volume  \
0      2022-11-11 14:38:00 -16540.10 16540.10 16486.50 16486.50 304.95   
1      2022-11-11 14:39:00  16489.30 16493.30 16456.20 16462.10 319.26   
2      2022-11-11 14:40:00  16465.50 16496.00 16447.40 16492.90 290.90   
3      2022-11-11 14:41:00  16492.90 16547.10 16472.50 16543.00 314.05   
4      2022-11-11 14:42:00  16540.20 16547.10 16498.20 16498.30 226.95   
...                    ...       ...      ...      ...      ...    ...   
67286  2022-12-29 23:55:00  16626.70 16626.70 16626.70 16626.70   2.35   
67287  2022-12-29 23:56:00  16626.70 16626.80 16626.70 16626.80   0.71   
67288  2022-12-29 23:57:00  16626.70 16626.80 16626.70 16626.80   2.61   
67289  2022-12-29 23:58:00  16626.70 16626.80 16626.70 16626.80   5.22   
67290  2022-12-29 23:59:00  16626.70 16626.80 16625.60 16625.60   8.01   

      Rec_count  Avg_price  ATR      OBV  Gain  Loss  Ema_gain  Ema_loss   RS  \
0            97   16516.81  Na

Now let's check what we got:


In [8]:
df = pd.read_csv("btc.csv", low_memory=False)
df.head(20)

,Time,Open,High,Low,Close,Volume,Rec_count,Avg_price,ATR,OBV,Gain,Loss,Ema_gain,Ema_loss,RS,RSI_14,ADTV,MFI,MFV,ADL
0,2022-11-11 14:38:00,-16540.10,16540.10,16486.50,16486.50,304.951,97,16516.81,NaN,0.00,0.00,53.60,NaN,NaN,NaN,NaN,140.69,-1.00,-304.95,-304.95
1,2022-11-11 14:39:00,16489.30,16493.30,16456.20,16462.10,319.262,103,16473.84,NaN,-319.26,0.00,27.20,NaN,NaN,NaN,NaN,140.69,-0.68,-217.72,-522.67
2,2022-11-11 14:40:00,16465.50,16496.00,16447.40,16492.90,290.905,99,16478.64,NaN,-28.36,27.40,0.00,NaN,NaN,NaN,NaN,140.69,0.87,253.79,-268.88
3,2022-11-11 14:41:00,16492.90,16547.10,16472.50,16543.00,314.045,99,16516.50,NaN,285.69,50.10,0.00,NaN,NaN,NaN,NaN,140.69,0.89,279.53,10.65
4,2022-11-11 14:42:00,16540.20,16547.10,16498.20,16498.30,226.951,99,16527.09,NaN,58.74,0.00,41.90,NaN,NaN,NaN,NaN,140.69,-1.00,-226.02,-215.37
5,2022-11-11 14:43:00,16498.30,16548.80,16491.90,16499.80,219.362,96,16519.31,NaN,278.10,1.50,0.00,NaN,NaN,NaN,NaN,140.69,-0.72,-158.45,-373.82
6,2022-11-11 14:44:00,16502.30,16542.00,16502.30,16540.80,233.955,101,16526.21,NaN,512.05,38.50,0.00,NaN,NaN,NaN,NaN,140.69,0.94,219.81,-154.01
7,2022-11-11 14:45:00,-16540.70,16746.00,16540.50,16742.70,Error,112,16639.43,NaN,1696.05,202.00,0.00,NaN,NaN,NaN,NaN,140.69,0.97,1145.97,991.95
8,2022-11-11 14:46:00,16747.60,16829.40,16686.90,16829.40,1356.19,113,16760.05,NaN,3052.24,81.80,0.00,NaN,NaN,NaN,NaN,140.69,1.00,1356.19,2348.14
9,2022-11-11 14:47:00,16829.80,17285.20,16828.30,17212.90,2122.798,116,17060.89,NaN,5175.03,383.10,0.00,NaN,NaN,NaN,NaN,140.69,0.68,1450.97,3799.12


As we can see, several question marks appeared in the dataframe; those are missing values which may hinder our further analysis.

<div>So, how do we identify all those missing values and deal with them?</div> 

<b>How to work with missing data?</b>

Steps for working with missing data:

<ol>
    <li>Identify missing data</li>
    <li>Deal with missing data</li>
    <li>Correct data format</li>
</ol>


## Identify and handle missing values

### Identify missing values
#### Convert missing data to NaN
In the trading dataset, missing data comes with the negative values,strings and "?".
We replace missing data with NaN (Not a Number), Python's default missing value marker for reasons of computational speed and convenience. Here we use the function: 
 <pre>.replace(A, B, inplace = True) </pre>
to replace A by B.
<p><p><code>Inplace=True</code> is used to save all changes in our dataframe.


Using this <code>r'^([A-Za-z]|_)+$'</code> to replace all letters in our dataset. Also we use <pre>pandas.DataFrame.mask</pre> to replace values where the condition is True. For example:<pre>.mask(A, B, inplace=True)</pre> 


Also we can see that there is text in our dataset and open price sometimes is <0:


In [9]:
count = (df['Open']<0).sum()
print("Count of values that are < 0 in Column open:", count)

Count of values that are < 0 in Column open: 4719


In [10]:
# replace "?" to NaN
df.replace("?", np.nan, inplace = True)
# replace all text to NaN
df.replace(r'^([A-Za-z]|_)+$', np.NaN, regex=True, inplace = True)
#change data type for column 'open' 
df.astype({'Open': 'float'})
#replace negative values to NaN
df.Open.mask(df.Open<0, np.nan, inplace=True)
#make a new dataframe for calculating our accuracy in the future
acdf = df
df.head(20)

,Time,Open,High,Low,Close,Volume,Rec_count,Avg_price,ATR,OBV,Gain,Loss,Ema_gain,Ema_loss,RS,RSI_14,ADTV,MFI,MFV,ADL
0,2022-11-11 14:38:00,NaN,16540.10,16486.50,16486.50,304.951,97,16516.81,NaN,0.00,0.00,53.60,NaN,NaN,NaN,NaN,140.69,-1.00,-304.95,-304.95
1,2022-11-11 14:39:00,16489.30,16493.30,16456.20,16462.10,319.262,103,16473.84,NaN,-319.26,0.00,27.20,NaN,NaN,NaN,NaN,140.69,-0.68,-217.72,-522.67
2,2022-11-11 14:40:00,16465.50,16496.00,16447.40,16492.90,290.905,99,16478.64,NaN,-28.36,27.40,0.00,NaN,NaN,NaN,NaN,140.69,0.87,253.79,-268.88
3,2022-11-11 14:41:00,16492.90,16547.10,16472.50,16543.00,314.045,99,16516.50,NaN,285.69,50.10,0.00,NaN,NaN,NaN,NaN,140.69,0.89,279.53,10.65
4,2022-11-11 14:42:00,16540.20,16547.10,16498.20,16498.30,226.951,99,16527.09,NaN,58.74,0.00,41.90,NaN,NaN,NaN,NaN,140.69,-1.00,-226.02,-215.37
5,2022-11-11 14:43:00,16498.30,16548.80,16491.90,16499.80,219.362,96,16519.31,NaN,278.10,1.50,0.00,NaN,NaN,NaN,NaN,140.69,-0.72,-158.45,-373.82
6,2022-11-11 14:44:00,16502.30,16542.00,16502.30,16540.80,233.955,101,16526.21,NaN,512.05,38.50,0.00,NaN,NaN,NaN,NaN,140.69,0.94,219.81,-154.01
7,2022-11-11 14:45:00,NaN,16746.00,16540.50,16742.70,NaN,112,16639.43,NaN,1696.05,202.00,0.00,NaN,NaN,NaN,NaN,140.69,0.97,1145.97,991.95
8,2022-11-11 14:46:00,16747.60,16829.40,16686.90,16829.40,1356.19,113,16760.05,NaN,3052.24,81.80,0.00,NaN,NaN,NaN,NaN,140.69,1.00,1356.19,2348.14
9,2022-11-11 14:47:00,16829.80,17285.20,16828.30,17212.90,2122.798,116,17060.89,NaN,5175.03,383.10,0.00,NaN,NaN,NaN,NaN,140.69,0.68,1450.97,3799.12


#### Evaluating for Missing Data

The missing values are converted by default. We use the following functions to identify these missing values. There are two methods to detect missing data:

<ol>
    <li><b>.isnull()</b></li>
    <li><b>.notnull()</b></li>
</ol>
The output is a boolean value indicating whether the value that is passed into the argument is in fact missing data.


In [11]:
missing_data = df.isnull()
missing_data.head(5)

,Time,Open,High,Low,Close,Volume,Rec_count,Avg_price,ATR,OBV,Gain,Loss,Ema_gain,Ema_loss,RS,RSI_14,ADTV,MFI,MFV,ADL
0,False,True,False,False,False,False,False,False,True,False,False,False,True,True,True,True,False,False,False,False
1,False,False,False,False,False,False,False,False,True,False,False,False,True,True,True,True,False,False,False,False
2,False,False,False,False,False,False,False,False,True,False,False,False,True,True,True,True,False,False,False,False
3,False,False,False,False,False,False,False,False,True,False,False,False,True,True,True,True,False,False,False,False
4,False,False,False,False,False,False,False,False,True,False,False,False,True,True,True,True,False,False,False,False


"True" means the value is a missing value while "False" means the value is not a missing value.


<h4>Count missing values in each column</h4>
<p>
Using a for loop in Python, we can quickly figure out the number of missing values in each column. As mentioned above, "True" represents a missing value and "False" means the value is present in the dataset.  In the body of the for loop the method ".value_counts()" counts the number of "True" values. 
</p>


In [12]:
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")    

Time
False    67291
Name: Time, dtype: int64

Open
False    62572
True      4719
Name: Open, dtype: int64

High
False    67291
Name: High, dtype: int64

Low
False    67291
Name: Low, dtype: int64

Close
False    67291
Name: Close, dtype: int64

Volume
False    62534
True      4757
Name: Volume, dtype: int64

Rec_count
False    62637
True      4654
Name: Rec_count, dtype: int64

Avg_price
False    67291
Name: Avg_price, dtype: int64

ATR
False    67277
True        14
Name: ATR, dtype: int64

OBV
False    67291
Name: OBV, dtype: int64

Gain
False    67291
Name: Gain, dtype: int64

Loss
False    67291
Name: Loss, dtype: int64

Ema_gain
False    67277
True        14
Name: Ema_gain, dtype: int64

Ema_loss
False    67277
True        14
Name: Ema_loss, dtype: int64

RS
False    67277
True        14
Name: RS, dtype: int64

RSI_14
False    67277
True        14
Name: RSI_14, dtype: int64

ADTV
False    67291
Name: ADTV, dtype: int64

MFI
False    67179
True       112
Name: MFI, dtype: int64

MFV

Based on the summary above, each column has 67291 rows of data and 11 of the columns containing missing data:

<ol>
    <li>"Open": 4651 missing data</li>
    <li>"Volume": 4651 missing data</li>
    <li>"Rec_count": 4651 missing data</li>
    <li>"ATR": 14 missing data</li>
    <li>"Ema_gain": 14 missing data</li>
    <li>"Ema_loss": 14 missing data</li>
    <li>"RS": 14 missing data</li>
    <li>"RSI_14": 14 missing data</li>
    <li>"ADL": 112 missing data</li>
    <li>"MFI": 112 missing data</li>
    <li>"MFV": 112 missing data</li>
</ol>


### Deal with missing data
<b>How to deal with missing data?</b>

<ol>
    <li>Drop data<br>
        a. Drop the whole row<br>
        b. Drop the whole column
    </li>
    <li>Replace data<br>
        a. Replace it by mean<br>
        b. Replace it by frequency<br>
        c. Replace it based on other functions
    </li>
</ol>


<b>What is the interpolation?</b>

<p>Interpolation is a process of calculating values based on known values. The interpolation is used to obtain intermediate values of properties, such as height, width, etc., during the process of their animation.</p>
<b>Why we will use the interpolation?</b>
<p><p>Because all our dataset is related to temporal changes, so for columns "Open" and "Volume" we will use interpolation.</p>
<b>Replace by frequency:</b>

<ul>
    <li>"Rec_count": +-4651 missing data, replace them with frequency</li>
</ul>
<b>Replace by interpolating:</b>

<ul>
    <li>"Open": +-4651 missing data, replace them with interpolating</li>
    <li>"Volume": +-4651 missing data, replace them with interpolating</li>
</ul>
<b>Drop the whole row:</b>

<ul>
    <li>"RS": +-4651 missing data, drop the whole row because the missing data is on the head of dataframe</li>
    <li>"RSI_14": +-4651 missing data,  drop the whole row because the missing data is on the head of dataframe</li>
    <li>"ATR": 14 missing data,, drop the whole row because the missing data is on the head of dataframe</li>
    <li>"Ema_gain": 14 missing data, drop the whole row because the missing data is on the head of dataframe</li>
    <li>"Ema_loss": 14 missing data, drop the whole row because the missing data is on the head of dataframe</li>
</ul>


#### Calculate the freq value for the "Rec_count" column 


To see which values are present in a particular column, we can use the ".value_counts()" method:


In [13]:
df['Rec_count'].value_counts()

25     1366
27     1293
23     1289
29     1265
26     1259
       ... 
118       2
115       2
119       2
122       1
124       1
Name: Rec_count, Length: 122, dtype: int64

We can see that '25' are the most common type. We can also use the ".idxmax()" method to calculate the most common type automatically:


In [14]:
s = df['Rec_count'].value_counts().idxmax()
s

'25'

#### Replace "NaN" with freq value in "Rec_count" column


In [15]:
df["Rec_count"].replace(np.nan, s, inplace=True)
df.head(20)

,Time,Open,High,Low,Close,Volume,Rec_count,Avg_price,ATR,OBV,Gain,Loss,Ema_gain,Ema_loss,RS,RSI_14,ADTV,MFI,MFV,ADL
0,2022-11-11 14:38:00,NaN,16540.10,16486.50,16486.50,304.951,97,16516.81,NaN,0.00,0.00,53.60,NaN,NaN,NaN,NaN,140.69,-1.00,-304.95,-304.95
1,2022-11-11 14:39:00,16489.30,16493.30,16456.20,16462.10,319.262,103,16473.84,NaN,-319.26,0.00,27.20,NaN,NaN,NaN,NaN,140.69,-0.68,-217.72,-522.67
2,2022-11-11 14:40:00,16465.50,16496.00,16447.40,16492.90,290.905,99,16478.64,NaN,-28.36,27.40,0.00,NaN,NaN,NaN,NaN,140.69,0.87,253.79,-268.88
3,2022-11-11 14:41:00,16492.90,16547.10,16472.50,16543.00,314.045,99,16516.50,NaN,285.69,50.10,0.00,NaN,NaN,NaN,NaN,140.69,0.89,279.53,10.65
4,2022-11-11 14:42:00,16540.20,16547.10,16498.20,16498.30,226.951,99,16527.09,NaN,58.74,0.00,41.90,NaN,NaN,NaN,NaN,140.69,-1.00,-226.02,-215.37
5,2022-11-11 14:43:00,16498.30,16548.80,16491.90,16499.80,219.362,96,16519.31,NaN,278.10,1.50,0.00,NaN,NaN,NaN,NaN,140.69,-0.72,-158.45,-373.82
6,2022-11-11 14:44:00,16502.30,16542.00,16502.30,16540.80,233.955,101,16526.21,NaN,512.05,38.50,0.00,NaN,NaN,NaN,NaN,140.69,0.94,219.81,-154.01
7,2022-11-11 14:45:00,NaN,16746.00,16540.50,16742.70,NaN,112,16639.43,NaN,1696.05,202.00,0.00,NaN,NaN,NaN,NaN,140.69,0.97,1145.97,991.95
8,2022-11-11 14:46:00,16747.60,16829.40,16686.90,16829.40,1356.19,113,16760.05,NaN,3052.24,81.80,0.00,NaN,NaN,NaN,NaN,140.69,1.00,1356.19,2348.14
9,2022-11-11 14:47:00,16829.80,17285.20,16828.30,17212.90,2122.798,116,17060.89,NaN,5175.03,383.10,0.00,NaN,NaN,NaN,NaN,140.69,0.68,1450.97,3799.12


#### Fill all NaN in the "Open" column by interpolating


Check which method is better for our dataset:


In [16]:
#make DataFrame for result
res = pd.DataFrame({"Method":[], "Accuracy":[], "Percent(%)":[]})

#all methods
methods = ['linear', 'nearest', 'quadratic', 'cubic', 'from_derivatives', 'piecewise_polynomial', 'pchip', 'akima', 'cubicspline']
#methods with order 
other = ['polynomial','slinear']

#test all methods for func 'pabdas.interpolate'
for method in methods:
    #check in which rows were NaN values
    rows = acdf["Open"].isna()
    #use one method
    interpolation_check = df["Open"].interpolate(method = method)
    # calculate accuracy by 'mean_squared_error[ and percent of accuracy by 'mean_absolute_percentage_error'
    accuracy = mean_squared_error(newdf.loc[rows, "Open"], interpolation_check[rows])
    percent = mean_absolute_percentage_error(newdf.loc[rows, "Open"], interpolation_check[rows])
    # write a result in our dataframe
    res.loc[len(res.index)] = [method, accuracy, percent*100]
for method in other:
    for order in [3,7]:
        rows = acdf["Open"].isna()
        interpolation_check = df["Open"].interpolate(method = method, order=order)
        accuracy = mean_squared_error(newdf.loc[rows, "Open"], interpolation_check[rows])
        percent = mean_absolute_percentage_error(newdf.loc[rows, "Open"], interpolation_check[rows])
        res.loc[len(res.index)] = [f'{method}_{order}', accuracy, percent*100]

#set precision 15
pd.options.display.float_format = '{:.15f}'.format
#let's look at our result
res.sort_values(by = "Accuracy", ascending = True).head(20)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

So we will use the method 'from_derivatives':


In [ ]:
df["Open"].interpolate(method = 'from_derivatives', inplace=True)

Now let's check if the values have changed:


In [ ]:
missing_data = df.isnull()
print("Open")
print (missing_data["Open"].value_counts())
print("")    

The replacement procedure is very similar to what we have seen previously:


```python
#replace the missing 'col1' values by the most frequent 
df["col1"].replace(val1, val2, inplace=True)
```


### Drop the whole row


Also we can drop rows, where we can't change NaN. For example:

```python 
# simply drop whole row with NaN in "col1" column
df.dropna(subset=["col1"], axis=0, inplace=True)

# reset index, because we droped some rows
df.reset_index(drop=True, inplace=True)
```


In [ ]:
# simply drop whole row with NaN in columns
df.dropna(subset=["ATR", "Ema_gain","Ema_loss", "RS","RSI_14"], axis=0, inplace=True)

# reset index, because we droped some rows
df.reset_index(drop=True, inplace=True)

Let's check what we got:


In [ ]:
missing_data = df.isnull()
print("ATR")
print (missing_data["ATR"].value_counts())
print("")    

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #1 a): </h1>

<b>Based on the example above, replace NaN in "Volume" column with the interpolation and mathod 'linear'.</b>

</div>


In [ ]:
# Write your code below and press Shift+Enter to execute


<details><summary>Click here for the solution</summary>

```python
#replace Nan in column volumn by method linear 
df["volume"].interpolate(method = 'linear', inplace=True)
df.head()
```

</details>


<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #1 b): </h1>

<b>Drop the whole rows where is NaN for columns "ADL","MFI","MFV".

</div>


In [ ]:
# Write your code below and press Shift+Enter to execute


<details><summary>Click here for the solution</summary>

```python
# simply drop whole row with NaN in "col1" column
df.dropna(subset=["ADL","MFI","MFV"], axis=0, inplace=True)

# reset index, because we droped some rows
df.reset_index(drop=True, inplace=True)
```

</details>


In [ ]:
# set precision 2
pd.options.display.float_format = '{:.2f}'.format
df.head()

<b>Good!</b> Now, we have a dataset with no missing values.


### Correct data format
<b>We are almost there!</b>
<p>The last step in data cleaning is checking and making sure that all data is in the correct format (int, float, text or other).</p>

In Pandas, we use:

<p><b>.dtype()</b> to check the data type</p>
<p><b>.astype()</b> to change the data type</p>


#### Let's list the data types for each column


In [ ]:
df.dtypes

<p>As we can see above, some columns are not of the correct data type. Numerical variables should have type 'float' or 'int', and variables with strings such as categories should have type 'object'. For example, 'volume' and 'rec_count' variables are numerical values that describe the trade, so we should expect them to be of the type 'float' or 'int'; however, they are shown as type 'object'. We have to convert data types into a proper format for each column using the "astype()" method.</p> 


#### Convert data types to proper format 


In [ ]:
df[["Volume"]] = df[["Volume"]].astype("float")
df[["Time"]] = df[["Time"]].astype("datetime64")
df[["Rec_count"]] = df[["Rec_count"]].astype("int")

#### Let us list the columns after the conversion


In [ ]:
df.dtypes

<b>Wonderful!</b>

Now we have finally obtained the cleaned dataset with no missing values with all data in its proper format.


## Data Standardization
<p>
Data is usually collected from different agencies in different formats.
(Data standardization is also a term for a particular type of data normalization where we subtract the mean and divide by the standard deviation.)
</p>

<b>What is standardization?</b>

<p>Standardization is the process of transforming data into a common format, allowing the researcher to make the meaningful comparison.
</p>

<b>Example</b>

<p>Transform USDT to EUR:</p>
<p>In our dataset, the fuel consumption columns "avg_price","open", "close","high" and "low" are represented by BUSD (BUSD per BTC). Assume we are developing an application in a country that accepts the EUR standard.</p>
<p>We will need to apply <b>data transformation</b> to transform BUSD into EUR.</p>


<p>The formula for unit conversion is:<p>
1 USDT = 0.92 EUR  (24.01.2023)
<p>We can do many mathematical operations directly in Pandas.</p>


In [ ]:
# Convert mpg to EUR by mathematical operation
res = requests.get("https://api.binance.com/sapi/v1/convert/exchangeInfo?fromAsset=BUSD&toAsset=EUR")
if res.status_code != 200:
    rate = 0.92
else:
    res = res.json()
    rate = float(res[0]["toAssetMinAmount"])
    
print(f"The exchange rate is 1 BUSD = {rate} EUR")


In [ ]:
# Convert USDT to EUR by mathematical operation (0.92 * price)
df['avg_price_eur'] = rate*df["Avg_price"]
df['open_eur'] = rate*df["Open"]
df['high_eur'] = rate*df["High"]
df['low_eur'] = rate*df["Low"]

# check your transformed data 
df.head()

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #2: </h1>

<b>According to the example above, transform BUSD to USDT in the column of "close",write data in the column "close_USDT".</b>

</div>


In [ ]:
# Write your code below and press Shift+Enter to execute 


<details><summary>Click here for the solution</summary>

```python
# Convert mpg to USDT by mathematical operation
res = requests.get("https://api.binance.com/sapi/v1/convert/exchangeInfo?fromAsset=BUSD&toAsset=USDT")
if res.status_code != 200:
    rate = 0.9971
else:
    res = res.json()
    rate = float(res[0]["toAssetMinAmount"])
    
print(f"The exchange rate is 1 BUSD = {rate} USDT")

df['close_USDT'] = rate*df["Close"]

```

</details>


## Data Normalization

<b>Why normalization?</b>

<p>Normalization is the process of transforming values of several variables into a similar range. Typical normalizations include scaling the variable so the variable average is 0, scaling the variable so the variance is 1, or scaling the variable so the variable values range from 0 to 1.
</p>

<b>Example</b>

<p>To demonstrate normalization, let's say we want to scale the columns "volume" and "rec_count".</p>
<p><b>Target:</b> would like to normalize those variables so their value ranges from 0 to 1</p>
<p><b>Approach:</b> create a new column with value by sklearn.preprocessing.MinMaxScaler</p>


In [ ]:
# Normalization with sklearn.preprocessing.MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df[['Volume','Rec_count']])
df[['volume_norm','rec_counter_norm']]=scaler.transform(df[['Volume','Rec_count']])
print(df[['volume_norm','rec_counter_norm']])

Also you can use sklearn.preprocessing.Normalizer:
```python
#second method to normalize columns
from sklearn.preprocessing import Normalizer

transformer = Normalizer().fit(dataframe[['col1','col2']])  # fit does nothing.
transformer.transform(dataframe[['col1','col2']])
```


In [ ]:
df.head()

Here we can see we've normalized "volume" and "rec_count" in the range of \[0,1].


<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question #3: </h1>

<b>According to the example above, normalize the columns "Volume" and "Rec_count" by second way.</b>

</div>


In [ ]:
# Write your code below and press Shift+Enter to execute 


<details><summary>Click here for the solution</summary>

```python
#second method to normalize columns
from sklearn.preprocessing import Normalizer

transformer = Normalizer().fit(dataframe[['Volume','Rec_count']])  # fit does nothing.
transformer.transform(dataframe[['Volume','Rec_count']])
```

</details>


## Binning
<b>Why binning?</b>
<p>
    Binning is a process of transforming continuous numerical variables into discrete categorical 'bins' for grouped analysis.
</p>

<b>Example: </b>

<p>In our dataset, "Open" is a real valued variable ranging from 15478.5 to 18344.6 and it has 17113 unique values. What if we only care about the open price difference between trades with high open, medium open, and little open (3 types)? Can we rearrange them into three ‘bins' to simplify analysis? </p>

<p>We will use the pandas method 'cut' to segment the 'Open' column into 3 bins.</p>


### Example of Binning Data In Pandas


Let's plot the histogram of open to see what the distribution of open looks like.


In [ ]:
plt.pyplot.hist(df["Open"])
# set x/y labels and plot title
plt.pyplot.xlabel("open")
plt.pyplot.ylabel("price")
plt.pyplot.title("open bins")

<p>We would like 3 bins of equal size bandwidth so we use numpy's <code>linspace(start_value, end_value, numbers_generated</code> function.</p>
<p>Since we want to include the minimum value of open, we want to set start_value = min(df["Open"]).</p>
<p>Since we want to include the maximum value of open, we want to set end_value = max(df["Open"]).</p>
<p>Since we are building 3 bins of equal length, there should be 4 dividers, so numbers_generated = 4.</p>


We build a bin array with a minimum value to a maximum value by using the bandwidth calculated above. The values will determine when one bin ends and another begins.


In [ ]:
bins = np.linspace(min(df["Open"]), max(df["Open"]), 4)
bins

We set group  names:


In [ ]:
group_names = ['Low', 'Medium', 'High']

We apply the function "cut" to determine what each value of `df['Open']` belongs to.


In [ ]:
df['open-binned'] = pd.cut(df['Open'], bins, labels=group_names, include_lowest=True )
df[['Open','open-binned']].head(20)

Let's see the number of values in each bin:


In [ ]:
df["open-binned"].value_counts()

Let's plot the distribution of each bin:


In [ ]:
# sort group names by number of values
group_names = ['Medium','Low', 'High']
pyplot.bar(group_names, df["open-binned"].value_counts())

# set x/y labels and plot title
plt.pyplot.xlabel("open")
plt.pyplot.ylabel("count")
plt.pyplot.title("open bins")

<p>
    Look at the dataframe above carefully. You will find that the last column provides the bins for "Open" based on 3 categories ("Low", "Medium" and "High"). 
</p>
<p>
    We successfully narrowed down the intervals from 17113 to 3!
</p>


### Bins Visualization
Normally, a histogram is used to visualize the distribution of bins we created above. 


In [ ]:
# draw historgram of attribute "open" with bins = 3
plt.pyplot.hist(df["Open"], bins = 3)

# set x/y labels and plot title
plt.pyplot.xlabel("open")
plt.pyplot.ylabel("count")
plt.pyplot.title("open bins")

The plot above shows the binning result for the attribute "Open".


## Indicator Variable (or Dummy Variable)
<b>What is an indicator variable?</b>
<p>
    An indicator variable (or dummy variable) is a numerical variable used to label categories. They are called 'dummies' because the numbers themselves don't have inherent meaning. 
</p>

<b>Why we use indicator variables?</b>

<p>
    We use indicator variables so we can use categorical variables for regression analysis in the later modules.
</p>
<b>Example</b>
<p>
    We see the column "open-binned" has two unique values: "Low","Medium" and "High". Regression doesn't understand words, only numbers. To use this attribute in regression analysis, we convert "open-binned" to indicator variables.
</p>

<p>
    We will use pandas' method 'get_dummies' to assign numerical values to different categories of open binned. 
</p>


In [ ]:
df.columns

Get the indicator variables and assign it to data frame "dummy_variable\_1":


In [ ]:
dummy_variable_1 = pd.get_dummies(df["open-binned"])
dummy_variable_1.head()

Change the column names for clarity:


In [ ]:
dummy_variable_1.rename(columns={'Low':'Low-open-price', 'Medium':'Medium-open-price','High':'High-open-price'}, inplace=True)
dummy_variable_1.head()

In the dataframe, column 'open-binned' has values for 'Low','Medium and 'High' as 0s and 1s now.


In [ ]:
# merge data frame "df" and "dummy_variable_1" 
df = pd.concat([df, dummy_variable_1], axis=1)

# drop original column "open-binned" from "df"
df.drop("open-binned", axis = 1, inplace=True)

In [ ]:
df.head()

The last three columns are now the indicator variable representation of the open-binned variable. They're all 0s and 1s now.


<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #4: </h1>

<b>Similar to before, create an indicator variable for the column "Time" by days of week with func "day_name()"</b>

</div>


In [ ]:
# Write your code below and press Shift+Enter to execute 


<details><summary>Click here for the solution</summary>

```python
#get name of day 
s = df.Time
#write in a new column 'day_name'
df['day_name'] = s.dt.day_name()

# get indicator variables of rec_count and assign it to data frame "dummy_variable_2"
dummy_variable_2 = pd.get_dummies(df['day_name'])

# show first 5 instances of data frame "dummy_variable_2"
dummy_variable_2.head()

```

</details>


## Resample time-series data.


<b>What is a pandas.DataFrame.resample?</b>

<p>Convenience method for frequency conversion and resampling of time series.
The object must have a datetime-like index (<cite>DatetimeIndex</cite>, <cite>PeriodIndex</cite>,
or <cite>TimedeltaIndex</cite>), or the caller must pass the label of a datetime-like
series/index to the <code class="docutils literal notranslate"><span class="pre">on</span></code>/<code class="docutils literal notranslate"><span class="pre">level</span></code> keyword parameter.</p>

<b>Why we use a pandas.DataFrame.resample?</b>

<p>We use this function to get for some period summary data. </p>


Make new dataset:


In [ ]:
series = pd.DataFrame()

Set column 'Time' as index:


In [ ]:
df['Time'] = pd.to_datetime(df['Time'])
df.set_index('Time', inplace=True)

Find our dataframe per day:


In [ ]:
series['Open'] = df['Open'].resample('D').first()
series['High'] = df['High'].resample('D').max()
series['Low'] = df['Low'].resample('D').min()
series['Close'] = df['Close'].resample('D').last()
series['Volume'] = df['Volume'].resample('D').sum()
series.head()

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question #5: </h1>

<b>Make as in the example, but every 1 hour.</b>

</div>


In [ ]:
# Write your code below and press Shift+Enter to execute


<details><summary>Click here for the solution</summary>

```python

task = pd.DataFrame()
task['Open'] = df['Open'].resample('1H').first()
task['High'] = df['High'].resample('1H').max()
task['Low'] = df['Low'].resample('1H').min()
task['Close'] = df['Close'].resample('1H').last()
task['Volume'] = df['Volume'].resample('1H').sum()
task.head()

```

</details>


Save the new csv:

> Note : The  csv file cannot be viewed in the jupyterlite based SN labs environment.However you can Click <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DA0101EN-SkillsNetwork/labs/Module%202/DA0101EN-2-Review-Data-Wrangling.ipynb?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDA0101ENSkillsNetwork20235326-2022-01-01">HERE</a> to download the lab notebook (.ipynb) to your local machine and view the csv file once the notebook is executed.


In [ ]:
df.to_csv('clean_df.csv')

### Thank you for completing this lab!

## Author

<a href="https://www.linkedin.com/in/joseph-s-50398b136/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDA0101ENSkillsNetwork20235326-2021-01-01">Joseph Santarcangelo</a>

### Other Contributors

<a href="https://www.linkedin.com/in/mahdi-noorian-58219234/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDA0101ENSkillsNetwork20235326-2021-01-01">Mahdi Noorian PhD</a>

Bahare Talayian

Eric Xiao

Steven Dong

Parizad

Hima Vasudevan

<a href="https://www.linkedin.com/in/fiorellawever/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDA0101ENSkillsNetwork20235326-2021-01-01">Fiorella Wenver</a>

<a href="https:// https://www.linkedin.com/in/yi-leng-yao-84451275/ " target="_blank" >Yi Yao</a>

<a href="https://www.linkedin.com/in/bohdan-tsisinskyi-539913255/ " target="_blank" >Bohdan Tsisinskyi</a>

<a href="https://author.skills.network/instructors/yaroslav_vyklyuk_2?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX0ZDWEN2351-2023-01-01">Prof. Yaroslav Vyklyuk, DrSc, PhD</a>

<a href="https://author.skills.network/instructors/mariya_fleychuk?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMSkillsNetworkGPXX0ZDWEN2351-2023-01-01">Prof. Mariya Fleychuk, DrSc, PhD</a>.

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description                  |
| ----------------- | ------- | ---------- | ----------------------------------- |
| 2023-03-04        | 1.0     | Bohdan     | Lab created      |


<hr>

## <h3 align="center"> © IBM Corporation 2023. All rights reserved. <h3/>
